<a href="https://colab.research.google.com/github/armindocachada/translate-video-gcp/blob/master/Wav2Lip_quick_trial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Collab preliminaries

In [2]:
!nvcc --version
!ffmpeg --help

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enabl

**Install dependencies for Google Colab**

In [14]:
!pip3 install google-cloud-pubsub==1.7.0
!pip3 install google-cloud-storage==1.29.0
!pip3 install setuptools>=40.3.0
!pip3 install google-cloud-speech
!pip3 install google-cloud-translate==2.0.1
!pip3 install google-cloud-texttospeech


     |████████████████████████████████| 153kB 8.5MB/s 
  Created wheel for grpc-google-iam-v1: filename=grpc_google_iam_v1-0.12.3-cp36-none-any.whl size=18500 sha256=23fc0836ffd9be9f7defcc7586bf25b22f13c5110f6fcdffa417b312345d7379
  Stored in directory: /root/.cache/pip/wheels/de/3a/83/77a1e18e1a8757186df834b86ce6800120ac9c79cd8ca4091b
Successfully built grpc-google-iam-v1


     |████████████████████████████████| 92kB 6.2MB/s 
     |████████████████████████████████| 92kB 6.3MB/s 
ERROR: google-cloud-bigquery 1.21.0 has requirement google-resumable-media!=0.4.0,<0.5.0dev,>=0.3.1, but you'll have google-resumable-media 0.5.1 which is incompatible.
ERROR: google-api-core 1.23.0 has requirement google-auth<2.0dev,>=1.21.1, but you'll have google-auth 1.17.2 which is incompatible.
  Found existing installation: google-resumable-media 0.4.1
    Uninstalling google-resumable-media-0.4.1:
      Successfully uninstalled google-resumable-media-0.4.1
  Found existing installation: google-api-core 1.16.0
    Uninstalling google-api-core-1.16.0:
      Successfully uninstalled google-api-core-1.16.0
  Found existing installation: google-cloud-core 1.0.3
    Uninstalling google-cloud-core-1.0.3:
      Successfully uninstalled google-cloud-core-1.0.3
  Found existing installation: google-cloud-storage 1.18.1
    Uninstalling google-cloud-storage-1.18.1:
      Successfull

     |████████████████████████████████| 112kB 8.9MB/s 
     |████████████████████████████████| 512kB 8.6MB/s 
     |████████████████████████████████| 51kB 7.1MB/s 
     |████████████████████████████████| 276kB 14.2MB/s 
     |████████████████████████████████| 122kB 19.7MB/s 
  Created wheel for proto-plus: filename=proto_plus-1.11.0-cp36-none-any.whl size=41571 sha256=6ed4e933e2fc373053cbc10a2d245deb582a19f86b88a76acd02f65c7a6dd47c
  Stored in directory: /root/.cache/pip/wheels/78/e3/6b/a14506581b1cde1ac1743f2939dcc06fc06a5af2aa224a334e
  Created wheel for pyyaml: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44619 sha256=4eb55e8ab9bcbbf261f6d0b806516b4e8cb9ae50b004f53e5195a8df8cfbbcb9
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
Successfully built proto-plus pyyaml
ERROR: google-colab 1.0.0 has requirement google-auth~=1.17.2, but you'll have google-auth 1.23.0 which is incompatible.
  Found existing installation: 

     |████████████████████████████████| 92kB 5.8MB/s 
  Found existing installation: google-cloud-translate 1.5.0
    Uninstalling google-cloud-translate-1.5.0:
      Successfully uninstalled google-cloud-translate-1.5.0


     |████████████████████████████████| 61kB 4.6MB/s 


**Initialisation Variables**

In [13]:
import os

service_account_file_path="/content/sample_data/wavlip-gcp-cc941dc32aa8.json" #@param {type:"string"}
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=service_account_file_path

language_code='ja-JP' #@param {type:"string"}
original_video="/content/sample_data/introduction_original.mp4"  #@param {type:"string"}

language = language_code.split("-")[0]
soundPath="/content/sample_data/{}.mp3".format(language)



In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Get the code and models

In [7]:
!git clone https://github.com/Rudrabha/Wav2Lip.git

Cloning into 'Wav2Lip'...
remote: Enumerating objects: 65, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 311 (delta 31), reused 6 (delta 0), pack-reused 246
Receiving objects: 100% (311/311), 503.21 KiB | 6.62 MiB/s, done.
Resolving deltas: 100% (159/159), done.


In [9]:
!ls /content/gdrive/MyDrive/wav2lip

wav2lip_gan.pth


In [11]:
!cp -ri "/content/gdrive/MyDrive/wav2lip/wav2lip_gan.pth" /content/Wav2Lip/checkpoints/

cp: overwrite '/content/Wav2Lip/checkpoints/wav2lip_gan.pth'? ^C


# Get the pre-requisites

In [ ]:
#!pip uninstall tensorflow tensorflow-gpu

In [12]:
!cd Wav2Lip && pip install -r requirements.txt

     |████████████████████████████████| 1.6MB 9.5MB/s 
     |████████████████████████████████| 20.4MB 7.9MB/s 
     |████████████████████████████████| 55.7MB 53kB/s 
     |████████████████████████████████| 26.6MB 126kB/s 
     |████████████████████████████████| 83.1MB 129kB/s 
     |████████████████████████████████| 676.9MB 22kB/s 
     |████████████████████████████████| 2.6MB 47.0MB/s 
     |████████████████████████████████| 61kB 9.5MB/s 
     |████████████████████████████████| 3.1MB 42.9MB/s 
     |████████████████████████████████| 51kB 8.3MB/s 
  Created wheel for librosa: filename=librosa-0.7.0-cp36-none-any.whl size=1598345 sha256=d229339f695abef4c06c63614319e75b1f34f709020ba3446600989ba565fd6b
  Stored in directory: /root/.cache/pip/wheels/49/1d/38/c8ad12fcad67569d8e730c3275be5e581bd589558484a0f881
Successfully built librosa
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement img

In [13]:
!wget "https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth" -O "Wav2Lip/face_detection/detection/sfd/s3fd.pth"

--2020-11-28 20:48:24--  https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth
Resolving www.adrianbulat.com (www.adrianbulat.com)... 45.136.29.207
Connecting to www.adrianbulat.com (www.adrianbulat.com)|45.136.29.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 89843225 (86M) [application/octet-stream]
Saving to: ‘Wav2Lip/face_detection/detection/sfd/s3fd.pth’

Wav2Lip/face_detect 100%[===================>]  85.68M  13.7MB/s    in 7.1s    

2020-11-28 20:48:32 (12.1 MB/s) - ‘Wav2Lip/face_detection/detection/sfd/s3fd.pth’ saved [89843225/89843225]



Define functions for translations

In [15]:

from google.cloud import storage
import io
import os
import subprocess
from google.cloud import speech

def extract_audio_ffmpeg(filePath):
    filePathOutput = os.path.splitext(filePath)[0] + '.mp3'
    print(filePathOutput)
    subprocess.call(['ffmpeg', '-i', filePath, filePathOutput])
    return filePathOutput


def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    # bucket_name = "your-bucket-name"
    # source_file_name = "local/path/to/file"
    # destination_blob_name = "storage-object-name"

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print(
        "File {} uploaded to {}.".format(
            source_file_name, destination_blob_name
        )
    )

#from google.cloud import speech_v1p1beta1


def speech_to_text(audio_file_path):


  # Instantiates a client
  client = speech.SpeechClient()

  # The name of the audio file to transcribe
  file_name = audio_file_path

  # Loads the audio into memory
  with io.open(file_name, "rb") as audio_file:
      content = audio_file.read()
      audio = speech.RecognitionAudio(content=content)

  config = speech.RecognitionConfig(
      encoding=speech.RecognitionConfig.AudioEncoding.ENCODING_UNSPECIFIED,
      sample_rate_hertz=44100,
      language_code="en-GB",
  )

  # Detects speech in the audio file
  response = client.recognize(config=config, audio=audio)
  transcription=""
  for result in response.results:
      print("Transcript: {}".format(result.alternatives[0].transcript))
      transcription += result.alternatives[0].transcript

  return transcription

def translate(text,language):
    from google.cloud import translate_v2 as translate
    translate_client = translate.Client()

    if isinstance(text, bytes):
        text = text.decode('utf-8')

    # Text can also be a sequence of strings, in which case this method
    # will return a sequence of results for each text.
    result = translate_client.translate(
        text, target_language=language)

    print(u'Text: {}'.format(result['input']))
    print(u'Translation: {}'.format(result['translatedText']))

    print(u'Detected source language: {}'.format(
        result['detectedSourceLanguage'])
    )
    return result['translatedText']

def text_to_speech(speak, languageCode, outputFilePath, speed=1.0, voice_name=None, effects_profile_id=None):
    """Synthesizes speech from the input string of text or ssml.
    Note: ssml must be well-formed according to:
        https://www.w3.org/TR/speech-synthesis/
    """
    from google.cloud import texttospeech

    # Instantiates a client
    client = texttospeech.TextToSpeechClient()

    # Set the text input to be synthesized
    synthesis_input = texttospeech.SynthesisInput(text=speak)

    # Build the voice request, select the language code ("en-US") and the ssml
    # voice gender ("neutral")
    voice = texttospeech.VoiceSelectionParams(
        language_code=languageCode, name=voice_name
    )

    # Select the type of audio file you want returned
    audio_config = texttospeech.AudioConfig(
        audio_encoding=texttospeech.AudioEncoding.LINEAR16,
        speaking_rate=speed,
        effects_profile_id=[effects_profile_id]
    )

    # Perform the text-to-speech request on the text input with the selected
    # voice parameters and audio file type
    response = client.synthesize_speech(
        input=synthesis_input, voice=voice, audio_config=audio_config
    )

    # The response's audio_content is binary.
    with open(outputFilePath, "wb") as out:
        # Write the response to the output file.
        out.write(response.audio_content)
        print('Audio content written to file "{}"'.format(outputFilePath))


# [START tts_list_voices]
def list_voices(language):
    """Lists the available voices."""
    from google.cloud import texttospeech
    voice_names=[]
    client = texttospeech.TextToSpeechClient()

    # Performs the list voices request
    voices = client.list_voices()

    for voice in voices.voices:
        # Display the voice's name. Example: tpc-vocoded
       if language in voice.language_codes:
         
          
          voice_names.append(voice.name)
          #print(f"Name: {voice.name}")

          # Display the supported language codes for this voice. Example: "en-US"
         # for language_code in voice.language_codes:
         #     print(f"Supported language: {language_code}")

          #ssml_gender = texttospeech.SsmlVoiceGender(voice.ssml_gender)

          # Display the SSML Voice Gender
          #print(f"SSML Voice Gender: {ssml_gender.name}")

          # Display the natural sample rate hertz for this voice. Example: 24000
          #print(f"Natural Sample Rate Hertz: {voice.natural_sample_rate_hertz}\n")
    return voice_names

**Initialise Parameters**



**Transcribe Audio to text**



In [14]:
audio_file_path = extract_audio_ffmpeg(original_video)
# upload_blob("translations_code_mental", audio_file_path, os.path.basename(audio_file_path))
transcript = speech_to_text(audio_file_path)
voices=list_voices(language_code)
print(f"voices available={voices}")

/content/sample_data/introduction_original.mp3
Transcript: hello everyone today I will show you how you can speak more than 33 languages with the Google Cloud life on
['de-DE']
ja-JP
['en-GB']
ja-JP
['en-IN']
ja-JP
['hi-IN']
ja-JP
['id-ID']
ja-JP
['ar-XA']
ja-JP
['ar-XA']
ja-JP
['ar-XA']
ja-JP
['cmn-CN']
ja-JP
['cmn-CN']
ja-JP
['cmn-CN']
ja-JP
['cmn-CN']
ja-JP
['cmn-TW']
ja-JP
['cmn-TW']
ja-JP
['cmn-TW']
ja-JP
['cs-CZ']
ja-JP
['da-DK']
ja-JP
['da-DK']
ja-JP
['da-DK']
ja-JP
['da-DK']
ja-JP
['de-DE']
ja-JP
['de-DE']
ja-JP
['de-DE']
ja-JP
['de-DE']
ja-JP
['de-DE']
ja-JP
['el-GR']
ja-JP
['en-AU']
ja-JP
['en-AU']
ja-JP
['en-AU']
ja-JP
['en-AU']
ja-JP
['en-GB']
ja-JP
['en-GB']
ja-JP
['en-GB']
ja-JP
['en-GB']
ja-JP
['en-IN']
ja-JP
['en-IN']
ja-JP
['en-IN']
ja-JP
['en-US']
ja-JP
['en-US']
ja-JP
['en-US']
ja-JP
['en-US']
ja-JP
['en-US']
ja-JP
['en-US']
ja-JP
['en-US']
ja-JP
['en-US']
ja-JP
['en-US']
ja-JP
['en-US']
ja-JP
['es-ES']
ja-JP
['fi-FI']
ja-JP
['fil-PH']
ja-JP
['fil-PH']
ja-JP
['fil-PH

**Text to Speech File**

In [19]:
from IPython.display import Audio

#@title Generate Speech Using a Specific Voice
#@markdown 
voice_name = 'ja-JP-Wavenet-C' #@param {type:"string"} 
device = "large-home-entertainment-class-device"  #@param ["wearable-class-device","handset-class-device","headphone-class-device","small-bluetooth-speaker-class-device","medium-bluetooth-speaker-class-device","large-home-entertainment-class-device","large-automotive-class-device","telephony-class-application"]
speed = 1 #@param {type:"slider", min:0.1, max:5, step:0.1}
transcript = "Hello everyone today I will show you how you can speak more than 33 languages with the Google Cloud Platform and the WavLip AI Model"
translation = translate(transcript, language)
text_to_speech(translation, language, soundPath, speed=speed, voice_name=voice_name, effects_profile_id=device)
Audio(soundPath)
# merge_video_with_audio_ffmpeg(original_video, "{}.mp3".format(language),"{}_{}.mp4".format( os.path.splitext(original_video)[0], language),start_time_audio="00:00:02")


Text: Hello everyone today I will show you how you can speak more than 33 languages with the Google Cloud Platform and the WavLip AI Model
Translation: 今日はみなさん、こんにちは。GoogleCloudPlatformとWavLipAIモデルで33以上の言語を話す方法を紹介します。
Detected source language: en
Audio content written to file "/content/sample_data/ja.mp3"


## Now lets try!

In [24]:
#!cd Wav2Lip && python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face $original_video --audio $soundPath

output_file_name= "/content/Wav2Lip/results/result_voice.mp4"
from IPython.display import HTML
from base64 import b64encode

mp4 = open(output_file_name,'rb').read()
decoded_vid = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f'<video width=800 controls><source src={decoded_vid} type="video/mp4"></video>')


In [ ]:
# use the "files" button on the left to download the result in the Wav2Lip/results/ folder.

## **Variations to try**


1.   Use more padding to include the chin region

In [119]:
!cd Wav2Lip && python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face $original_video --audio $soundPath --pads 0 20 0 0

mp4 = open(output_file_name,'rb').read()
decoded_vid = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f'<video width=800 controls><source src={decoded_vid} type="video/mp4"></video>')

/bin/bash: line 0: cd: Wav2Lip: No such file or directory


2.   Use resize_factor to reduce the video resolution, as there is a change you might get better results for lower resolution videos. Why? Because the model was trained on low resolution faces.

In [25]:
!cd Wav2Lip && python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face $original_video --audio $soundPath --resize_factor 2

mp4 = open(output_file_name,'rb').read()
decoded_vid = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f'<video width=800 controls><source src={decoded_vid} type="video/mp4"></video>')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 